
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022


# <span style="color:#006E7F">__TP1 - BASECALLING and QC__ <a class="anchor" id="data"></span>  
    
## <span style="color: #4CACBC;"> 1. Creating the folder, downloading data and so on</span>  

### <span style="color: #4CACBD;"> 1.1 Simulated clones </span>

Before starting, please download special data created for this practical training. Data are available on the  from I-Trop server.

Each participant will analyse a alguea

To generate Clone data, a **1Mb** contig was extracted from chromosome 1 of rice.

20 levels of variation were generated and long reads were simulated for each.

We have introduced different variations (SNP, indel, indel+translocations) and also some contaminations. 

In [1]:
CLONE=Clone10

In [5]:
cd ~/work
mkdir -p DATA
cd DATA
# download your compressed CloneX 
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/${CLONE}.tar.gz

--2022-11-08 13:38:40--  https://itrop.ird.fr/ont-training/Clone10.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197949883 (189M) [application/x-gzip]
Saving to: ‘Clone10.tar.gz’

Clone10.tar.gz      100%[===================>] 188.78M  8.09MB/s    in 21s     

2022-11-08 13:39:02 (8.82 MB/s) - ‘Clone10.tar.gz’ saved [197949883/197949883]

FINISHED --2022-11-08 13:39:02--
Total wall clock time: 22s
Downloaded: 1 files, 189M in 21s (8.82 MB/s)


In [6]:
#decompress file
tar zxvf ${CLONE}.tar.gz

Clone10/
Clone10/ILL/
Clone10/ILL/Clone10_R1.fastq.gz
Clone10/ILL/Clone10_R2.fastq.gz
Clone10/reference.fasta
Clone10/ONT/
Clone10/ONT/Clone10.fastq.gz
Clone10/ONT/Clone10_DeepSimu_sequencing_summary.txt


In [5]:
# check data 
cd ~/work/DATA
ls -l ${CLONE}

total 1024
drwxr-x--- 2 jovyan users    4096 Sep 13  2021 ILL
drwxr-x--- 2 jovyan users    4096 Sep 13  2021 ONT
-rw-r----- 1 jovyan users 1037013 Sep 13  2021 reference.fasta


### <span style="color: #4CACBD;"> 1.2 Real data </span>

Some steps in this practical training can not work in Clones dataset. A second dataset will be download.

These data were generated in this [paper.](https://www.biorxiv.org/content/10.1101/2021.07.04.451066v1.full)

Please decompress Hh dataset.


In [8]:
cd ~/work/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/real_Hh.tar.gz

--2022-11-08 13:40:20--  https://itrop.ird.fr/ont-training/real_Hh.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2039732473 (1.9G) [application/x-gzip]
Saving to: ‘real_Hh.tar.gz’

real_Hh.tar.gz      100%[===================>]   1.90G  28.8MB/s    in 2m 11s  

2022-11-08 13:42:31 (14.9 MB/s) - ‘real_Hh.tar.gz’ saved [2039732473/2039732473]

FINISHED --2022-11-08 13:42:31--
Total wall clock time: 2m 11s
Downloaded: 1 files, 1.9G in 2m 11s (14.9 MB/s)


In [9]:
#decompress file
tar zxvf real_Hh.tar.gz

real_Hh/
real_Hh/H_M1C132_1.fastq.gz
real_Hh/REFH_M1C132.fasta
real_Hh/Hh-ASSEMBLIES/
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_ASSEMBLY-CIRCULARISED.fasta
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_ASSEMBLY-CIRCULARISED.fasta.fai
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_ASSEMBLY-CIRCULARISED.fasta.mmi
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_POLISHING_RACON-CIRCULARISED.fasta
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_POLISHING_RACON-CIRCULARISED.fasta.fai
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_POLISHING_RACON-CIRCULARISED.fasta.mmi
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta.fai
real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta.mmi
real_Hh/Hh-ASSEMBLIES/SHASTA-STEP_ASSEMBLY-CIRCULARISED.fasta
real_Hh/Hh-ASSEMBLIES/SHASTA-STEP_ASSEMBLY-CIRCULARISED.fasta.fai
real_Hh/Hh-ASSEMBLIES/SHASTA-STEP_ASSEMBLY-CIRCULARISED.fasta.mmi
real_Hh/Hh-ASSEMBLIES/RAVEN-STEP_ASSEMBLY-CIRCULARISED.fasta
real_Hh/Hh-ASS

## <span style="color: #4CACBC;"> 2. Basecalling </span>  

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 

### <span style="color: #4CACBD;"> 2.1 Basecalling with Guppy  <span style="color:red"> (Don't run it! ) </span> </span>

Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Documentation about how to run Guppy on I-trop GPU can be found on https://bioinfo.ird.fr/index.php/tutorials-fr/gpu/


In [6]:
MODEL="dna_r9.4.1_450bps_hac.cfg"
INPUT="path/to/FAST5"
OUTPUT="/path/to/FASTQ"
echo "guppy_basecaller -c ${MODEL} -i ${INPUT} --recursive -s ${OUTPUT} --num_callers 8 --gpu_runners_per_device 8 --device auto --min_qscore 7 --compress_fastq"

guppy_basecaller -c dna_r9.4.1_450bps_hac.cfg -i path/to/FAST5 --recursive -s /path/to/FASTQ --num_callers 8 --gpu_runners_per_device 8 --device auto --min_qscore 7 --compress_fastq


## <span style="color: #4CACBC;"> 3. Quality Control on Long Reads </span>  


Calculating data quantity

In [7]:
cd ~/work/DATA
pwd

/home/jovyan/work/DATA


Calculating how many bases were sequenced

In [8]:
seqtk seq -A ${CLONE}/ONT/${CLONE}.fastq.gz | grep -v ">" |wc -m

166447283


What is the sequencing depth if clone lenght is about 1Mb ?

### <span style="color: #4CACBD;"> 3.1 Quality Control of FASTQ with Nanoplot (one clone --summary) </span>

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [9]:
NanoPlot --help

usage: NanoPlot [-h] [-v] [-t THREADS] [--verbose] [--store] [--raw] [--huge]
                [-o OUTDIR] [--no_static] [-p PREFIX] [--tsv_stats]
                [--info_in_report] [--maxlength N] [--minlength N]
                [--drop_outliers] [--downsample N] [--loglength]
                [--percentqual] [--alength] [--minqual N] [--runtime_until N]
                [--readtype {1D,2D,1D2}] [--barcoded] [--no_supplementary]
                [-c COLOR] [-cm COLORMAP]
                [-f [{png,jpg,jpeg,webp,svg,pdf,eps,json} [{png,jpg,jpeg,webp,svg,pdf,eps,json} ...]]]
                [--plots [{kde,hex,dot} [{kde,hex,dot} ...]]]
                [--legacy [{kde,dot,hex} [{kde,dot,hex} ...]]] [--listcolors]
                [--listcolormaps] [--no-N50] [--N50] [--title TITLE]
                [--font_scale FONT_SCALE] [--dpi DPI] [--hide_stats]
                (--fastq file [file ...] | --fasta file [file ...] | --fastq_rich file [file ...] | --fastq_minimal file [file ...] | --summary fi

Launch NanoPlot by your Clone. You can launch NanoPlot using summaries or fastq files.

In [10]:
# create a folder to save results
mkdir -p ~/work/RESULTS
cd ~/work/RESULTS

In [11]:
# run nanoplot 
time NanoPlot -t 1 --summary ../DATA/${CLONE}/ONT/${CLONE}_DeepSimu_sequencing_summary.txt --outdir NANOPLOT_${CLONE}




If you read this then NanoPlot 1.40.2 has crashed :-(
Please try updating NanoPlot and see if that helps...

If not, please report this issue at https://github.com/wdecoster/NanoPlot/issues
If you could include the log file that would be really helpful.
Thanks!



Traceback (most recent call last):
  File "/opt/conda/envs/nanoplot/bin/NanoPlot", line 10, in <module>
    sys.exit(main())
  File "/opt/conda/envs/nanoplot/lib/python3.7/site-packages/nanoplot/NanoPlot.py", line 109, in main
    plots = make_plots(datadf, settings)
  File "/opt/conda/envs/nanoplot/lib/python3.7/site-packages/nanoplot/NanoPlot.py", line 229, in make_plots
    settings=settings)
  File "/opt/conda/envs/nanoplot/lib/python3.7/site-packages/nanoplotter/timeplots.py", line 59, in time_plots
    settings=settings)
  File "/opt/conda/envs/nanoplot/lib/python3.7/site-packages/nanoplotter/timeplots.py", line 65, in violin_plots_over_time
    dfs['timebin'] = add_time_bins(dfs)
  File "/opt/conda/envs/nanoplot/lib

: 1

* NOTE: Clones are simulated data. Check stats on created NanoStats file. <span style="color:red"> it's normal ! it doesn't work! ) </span>

Check stats on created NanoStats file.

In [12]:
cat NANOPLOT_${CLONE}/NanoStats.txt

General summary:         
Active channels:                     1.0
Mean read length:               14,809.4
Mean read quality:                   8.3
Median read length:             12,867.0
Median read quality:                 8.3
Number of reads:                11,239.0
Read length N50:                22,325.0
STDEV read length:              10,772.2
Total bases:               166,442,438.0
Number, percentage and megabases of reads above quality cutoffs
>Q5:	11239 (100.0%) 166.4Mb
>Q7:	11235 (100.0%) 166.4Mb
>Q10:	1 (0.0%) 0.0Mb
>Q12:	0 (0.0%) 0.0Mb
>Q15:	0 (0.0%) 0.0Mb
Top 5 highest mean basecall quality scores and their read lengths
1:	10.0 (2124)
2:	9.8 (3858)
3:	9.7 (976)
4:	9.6 (2838)
5:	9.6 (3723)
Top 5 longest reads and their mean basecall quality score
1:	73691 (8.2)
2:	64567 (8.2)
3:	63953 (8.4)
4:	62885 (8.2)
5:	60746 (8.2)


* What do you think about data? 

* Estimate coverage.

* What about reads quality qscore?

### <span style="color: #4CACBD;"> 3.2 Quality Control of FASTQ with Nanoplot (Hh real data --fastq ) </span>


Check quality using NanoPlot over Hh real data fastq. This can take a while.

In [13]:
cd ~/work/RESULTS/
pwd

/home/jovyan/work/RESULTS


**The following Nanoplot will run for a few minutes**

In [14]:
time NanoPlot -t 8 --fastq ../DATA/real_Hh/H_M1C132_1.fastq.gz --outdir NANOPLOT_Hh


real	2m17.584s
user	2m6.949s
sys	0m1.493s


Observe report.html.

* what about this dataset?

### <span style="color: #4CACBD;"> 3.3 Compare reads QC statistics with NanoComp (Summaries dataset) </span>

Compare long reads sequencing datasets using **NanoComp**.

NanoComp compiles quality information in a useful html report.

You can launch NanoComp using summaries or fastq files.

In [15]:
NanoComp --help

usage: NanoComp [-h] [-v] [-t THREADS] [-o OUTDIR] [-p PREFIX] [--verbose]
                [--raw] [--store] [--tsv_stats] [--make_no_static]
                [--readtype {1D,2D,1D2}] [--maxlength N] [--minlength N]
                [--barcoded] [--split_runs TSV_FILE]
                [-f [{png,jpg,jpeg,webp,svg,pdf,eps,json} [{png,jpg,jpeg,webp,svg,pdf,eps,json} ...]]]
                [-n names [names ...]] [-c colors [colors ...]]
                [--plot {violin,box,ridge,false}] [--title TITLE] [--dpi DPI]
                (--fasta file [file ...] | --fastq files [files ...] | --fastq_rich file [file ...] | --summary files [files ...] | --bam files [files ...] | --ubam file [file ...] | --cram file [file ...] | --pickle file [file ...] | --feather file [file ...])

Compares long read sequencing datasets.

General options:
  -h, --help            show the help and exit
  -v, --version         Print version and exit.
  -t, --threads THREADS
                        Set the allowed number 


For the moment, we used only a Clone and simulated data. 

Please download a "summaries" obtained in paspalum real data. 

Download available 'Real_PSummaries' to compare them. 


In [16]:
cd ~/work/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/real_summaries.tar.gz

--2022-11-09 13:34:43--  https://itrop.ird.fr/ont-training/real_summaries.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223723277 (213M) [application/x-gzip]
Saving to: ‘real_summaries.tar.gz’

real_summaries.tar. 100%[===================>] 213.36M  7.56MB/s    in 30s     

2022-11-09 13:35:13 (7.16 MB/s) - ‘real_summaries.tar.gz’ saved [223723277/223723277]

FINISHED --2022-11-09 13:35:13--
Total wall clock time: 30s
Downloaded: 1 files, 213M in 30s (7.16 MB/s)


In [17]:
#decompress 
tar -zxvf real_summaries.tar.gz

real_summaries/
real_summaries/OLD_summary.txt
real_summaries/FF_summary.txt
real_summaries/BON_summary.txt


In [18]:
cd ~/work/DATA/real_summaries/
ls -lh  *txt

-rw-r----- 1 jovyan users 271M Sep 13  2021 BON_summary.txt
-rw-r----- 1 jovyan users 247M Sep 13  2021 FF_summary.txt
-rw-r----- 1 jovyan users 246M Sep 13  2021 OLD_summary.txt


Compare this 3 summaries files with NanoComp

In [19]:
time NanoComp --summary OLD_summary.txt FF_summary.txt BON_summary.txt --outdir ~/work/RESULTS/NANOCOMP-realsummaries


real	0m22.570s
user	0m14.254s
sys	0m1.404s


### <span style="color: #4CACBD;"> Others complemental tools </span>
 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

* FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong
